<a href="https://colab.research.google.com/github/shIsmael/DeepLearning/blob/main/Deep_Learning_Practical_Concepts_and_Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions:
1. Make a copy of this notebook by opening the "File" tab and selecting "Save a copy in Drive"
2. Close this tab and move to your copy of this notebook
3. Follow the written guides within this notebook
4. If instructed to, add your own code in the corresponding cell

After completing this notebook, you will have:
- A basic understanding of the iterative process for developing models
- An understanding of how to effectively partition your dataset for training, development, and testing
- An understanding of what bias and variance are
- An understanding of what regularization is, how to implement common regularization techniques, and the effect of regularization on training




## Basic Development Cycle
Since it's almost impossible to correctly guess the most optimal values for your hyperparameters, we'll be using an iterative process for finding the most optimal values. Generally, development in deep learning is a highly iterative process, where you start with an idea, manifest it in the form of code, experiment with it, and the use the results of said experiment to form a new idea or refine your existing idea! It's important to keep this basic cycle in mind when you start working on your first project after Chapter 5. 

For example, let's consider a neural network that suffers from low accuracy. In this case, you could try a larger architecture (more layers/nodes), train for a longer period of time, use more advanced optimization algorithms, or try a completely different neural network architecture (i.e. CNN or autoencoder, which we'll cover in later chapters!). After trying one or more of these, you find that the neural network now suffers from low accuracy on the test set, yet has very high accuracy on the training set. In this case, you could try a larger dataset, use regularization (we'll cover this shortly!), or use a more appropriate neural network architecture to address this issue. 


## Dataset Partitioning
The development cycle will go much more smoothly when you partition your datasets correctly. As you've seen in previous notebooks, we've been using around 70% of our dataset for training and saving the last 30% for testing, which is what was usually done in past eras. Presently, most datasets are split into training, development/cross-validation, and test sets. The training set is used to train different variations on a model, while the cross-validation set is used to see which of said variations performs best. After iterating long enough by training new models and using the cross-validation set as a metric to obtain a final model, you can then use the test set to evaluate it as an unbiased estimate for performance. 

As for how you should partition your dataset into training, cross-validation, and testing sets, it depends on how large your dataset is; For smaller datasets of around 10,000 examples, it's perfectly fine to use the rule-of-thumb of 60%/20%/20%. For larger datasets of, say, millions of examples, you should only be using small percentages of your data (<5-10%) for cross-validation and testing, since those sets are only used for iterative development and evaluation. 

Another common mistake is using mismatched training and cross-validation/test set distributions. Training on a training set that comes from a different distribution (i.e. the training set comes from an open-source dataset, while the test set was manually created) can result in lower performance than expected on the cross-validation/test set, which can in turn result in slower iteration through the development cycle. 

## Metrics: Bias and Variance
We can use *bias* and *variance* to contextualize exactly how a given neural network is fitting to its training set. High bias would refer to *underfitting*, in which a neural network has not trained long enough/does not a large enough training set to properly generalize to be able to make accurate predictions. In contrast, high variance would refer to *overfitting*, in which a neural network has trained for too long and therefore is only able to make highly accurate predictions on the training/cross-validation set, with low performance on the test set. 

To quantify what high variance is, let's consider a neural network with a training set error (for now, think of error as % of examples incorrectly classified) of 1% and cross-validation set error of 10%. In this case, comparatively lower accuracy on the cross-validation set is a sign of high variance (overfitting). As for what high variance looks like, consider a neural network with a training set error of 20% and a cross-validation set error of 22%. Despite the proximity of the two errors, we observe that the network wasn't really learning anything. Thus, this neural network suffers from high bias (underfitting). High bias *and* variance are usually signified by high test set error and even higher cross-validation set error (i.e. 15%/30%). 

In the past, there existed a "trade-off" of sorts between bias and variance. However, with the tools that we have now, we can simply increase the size of an architecture or gather more data to reduce bias or variance without hurting the other. 

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split # for partitioning our dataset

# for the sake of having an example, we'll use the dataset from "Shallow Neural Networks"
# don't worry too much about what this function does!
def load_planar_dataset(seed):
    
    np.random.seed(seed)
    
    m = 10000 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j
        
    X = X.T
    Y = Y.T

    return X, Y

X, Y = load_planar_dataset(1) # using an arbitrary random seed of 1
print(X.shape) # contains the inputs
print(Y.shape) # contains the true labels
X = X.T
Y = Y.T
print(X.shape)
print(Y.shape)
# we can now partition our dataset into training, validation, and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
X_test, X_val, Y_test, Y_val = train_test_split(X_test, X_test, test_size=0.5) # 70% goes to training, 15% goes to val, and the last 15% goes to testing

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

## Regularization
### Overview
Regularization is used when your network suffers from high variance (in other words, is overfitting). Reducing variance by collecting more data is an option, but it's not always possible. Therefore, we can use regularization to limit how complex our model can get by "penalizing" our weight matrices with a hyperparameter $\lambda$ to keep their values small, which helps in "simplifying" our network, since each node will have a smaller effect. 

$L_2$ regularization and dropout will be implemented in the below code cell. 
### $L_2$ Regularization
Consider the generalized cost function $J(W^{[1]}, b^{[1]}, \dots, W^{[L]}, b^{[L]}) = \frac{1}{m}\sum^{m}_{i=1}\mathcal{L}(\hat{y}^{(i)}, y^{(i)})$. 
To add $L_2$ regularization, we simply add the term $\frac{\lambda}{2m}\sum^{L}_{l=1} ||W^{[L]}||^2$, where $\lambda$ is the regularization hyperparameter and $||W^{[l]}||^2$ is the squared Frobenius norm (the Frobenius norm of a matrix is defined as $\sum^{n^{[l-1]}}_{i=1}\sum^{n^{[l]}}_{j=1} (W_{ij}^{[l]})^2$). Our cost function is now $J(W^{[1]}, b^{[1]}, \dots, W^{[L]}, b^{[L]}) = \frac{1}{m}\sum^{m}_{i=1}\mathcal{L}(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m}\sum^{L}_{l=1} ||W^{[L]}||^2$, and we compute $\frac{\partial J}{\partial W^{[l]}}$ as usual with backpropagation, but we add the regularization term $\frac{\lambda}{m}W^{[l]}$ afterwards. We can then update $W^{[l]}$ as previously defined. 
### $L_1$ Regularization
To implement $L_1$ regularization, we can simply add $\frac{\lambda}{2m}|W^{[l]}|$ to the cost function. If you end up using $L_1$ regularization, the weight matrices will become sparse (meaning that they'll have many zeros), which can help with model compression (we're unfortunately not covering model compression). However, $L_1$ generally doesn't have as great of an effect on variance as $L_2$ does. 
### Dropout
To implement dropout regularization, we'll essentially, for each node in each layer, set a probability that the node will be "eliminated" from the network for an iteration of training (Note that different probabilties can be assigned to different layers and that connections to and from eliminated nodes are also eliminated). By doing this, we'll train a different network every iteration that is diminished due to eliminating random nodes, which in turn reduces the dependency of each node on other nodes, thereby limiting how complex the network can be. 
### Data Augmentation
Another less commonly used technique for reducing variance is artificially augmenting your training set. For instance, if your training set was comprised of images, you could horizontally flip and randomly rotate each image to double the size of your training set. These new images don't add as much to the training set as independently collecting new images would, but they're relatively inexpensive to generate. 
### Early Stopping
If you plot the training error and the cross-validation error while training for a long period of time, you'll find that the errors will eventually diverge; The training error will approach zero, and the cross-validation error will tend away from the training error. With early stopping, you'd use the parameters around some point in time where the two errors were relatively close. However, early stopping is even less commonly used for regularization because it often results in undesired performance and due to the introduction of other regularization techniques that don't directly impact training. 
### Adversarial Training
Adversarial training is a fairly new regularization technique based on *adversarial examples* which are inputs (i.e. slightly modified images) that can fool a neural network into misclassifying them. By training on a mixture of normal and adversarial examples, regularization can be achieved (in the original article, it states that this type of regularization could perform better than dropout) and additional robustness to aversarial examples. Of course, you don't need to know how to implement or use this type of regularization.

In [ ]:
# importing the necessary libraries
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import scipy.io
from google.colab import files # for uploading the dataset we'll use

# firstly, let's define a function that loads a dataset for which we'll implement 3-layer neural networks to classify it!
# don't worry about understanding what this function does!
def load_2D_dataset():
    data = scipy.io.loadmat('data.mat')
    train_X = data['X'].T
    train_Y = data['y'].T
    test_X = data['Xval'].T
    test_Y = data['yval'].T

    plt.scatter(train_X[0, :], train_X[1, :], c=train_Y, s=40, cmap=plt.cm.Spectral);
    
    return train_X, train_Y, test_X, test_Y 

# when prompted, download the dataset from https://drive.google.com/drive/folders/1jj1Ws0jkEwM9F0xPdKUtA0r-RajFeqU4?usp=sharing and upload it! 
# you do not need to upload the dataset more than once in a single session (if you've closed and reopened this notebook, you need to upload it again)
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

train_X, train_Y, test_X, test_Y = load_2D_dataset() # loading our dataset, don't worry about how we're partitioning it

# then, we can define the necessary functions to define a model and for implementing L2 regularization and dropout, which you've seen in previous notebooks
# note that we'll be using a 3-layer neural network
def sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

def relu(x):
    s = np.maximum(0,x)
    return s
def initialize_parameters(layer_dims): # layer_dims contains a list with the dimensions of each layer in the network    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) / np.sqrt(layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

def forward_propagation(X, parameters): # where X is the input dataset and parameters is a dictionary containing {W1, b1,..., W3, b3}        
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return A3, cache

def backward_propagation(X, Y, cache): # where X is the input dataset, Y is the true label set, and cache is from forward_propagation()
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

def update_parameters(parameters, grads, learning_rate): # updates parameters using gradient descent, where parameters and grads are both dictionaries
    n = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for k in range(n):
        parameters["W" + str(k+1)] = parameters["W" + str(k+1)] - learning_rate * grads["dW" + str(k+1)]
        parameters["b" + str(k+1)] = parameters["b" + str(k+1)] - learning_rate * grads["db" + str(k+1)]
        
    return parameters

def predict(X, y, parameters): # where X is the dataset you wish to label and parameters is the parameters of the final model    
    m = X.shape[1]
    p = np.zeros((1,m), dtype = np.int)
    
    # Forward propagation
    a3, caches = forward_propagation(X, parameters)
    
    # convert probas to 0/1 predictions
    for i in range(0, a3.shape[1]):
        if a3[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0

    # print results

    #print ("predictions: " + str(p[0,:]))
    #print ("true labels: " + str(y[0,:]))
    print("Accuracy: "  + str(np.mean((p[0,:] == y[0,:]))))
    
    return p

def compute_cost(a3, Y): # where a3 is the output of forwardpropagation and Y is the true label set
    m = Y.shape[1]
    
    logprobs = np.multiply(-np.log(a3),Y) + np.multiply(-np.log(1 - a3), 1 - Y)
    cost = 1./m * np.nansum(logprobs)
    
    return cost

def plot_decision_boundary(model, X, y): # we'll use this to visualize the effects of regularization!
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)
    plt.show()


# let's define a 3-layer neural network without regularization
# we'll implement the hyperparameters for L2 and dropout, but keep them at a value at which they have no effect on our model
def model(X, Y, learning_rate = 0.3, num_iterations = 30000, print_cost = True, lambd = 0, keep_prob = 1):
    # keep_prob is the probability of keeping of a certain percentage of neurons
    # lambd is lambda, defined above    
    grads = {}
    costs = []                            # to keep track of the cost
    m = X.shape[1]                        # number of examples
    layers_dims = [X.shape[0], 20, 3, 1]
    
    # initialize parameters 
    parameters = initialize_parameters(layers_dims)

    # gradient descent
    for i in range(0, num_iterations):
        # forwardpropagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        if keep_prob == 1: 
            a3, cache = forward_propagation(X, parameters)
        elif keep_prob < 1:
            a3, cache = forward_propagation_with_dropout(X, parameters, keep_prob)
        
        # computing the cost
        if lambd == 0:
            cost = compute_cost(a3, Y)
        else:
            cost = compute_cost_with_regularization(a3, Y, parameters, lambd)
            
        # backpropagation
        assert(lambd==0 or keep_prob==1)    # it is possible to use both L2 regularization and dropout, 
                                            # but this assignment will only explore one at a time
        if lambd == 0 and keep_prob == 1: 
            grads = backward_propagation(X, Y, cache)
        elif lambd != 0:
            grads = backward_propagation_with_regularization(X, Y, cache, lambd)
        elif keep_prob < 1:
            grads = backward_propagation_with_dropout(X, Y, cache, keep_prob)
        
        # update parameters
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 10000 iterations
        if print_cost and i % 10000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
        if print_cost and i % 1000 == 0:
            costs.append(cost)
    
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (x1,000)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

# testing without regularization as a benchmark
parameters = model(train_X, train_Y, lambd = 0, keep_prob = 1)
print ("Without regularization, on the training set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("Without regularization, on the test set:")
predictions_test = predict(test_X, test_Y, parameters)

# next, we'll implement the functions necessary for L2 regularization
def compute_cost_with_regularization(A3, Y, parameters, lambd): # where A3 is the output of forwardpropagation
# the equation for implementing this is defined above
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    cross_entropy_cost = compute_cost(A3, Y) # This gives you the cross-entropy part of the cost
    
    L2_regularization_cost = (1/m)*(lambd/2)*(np.sum(np.square(W1))+np.sum(np.square(W2))+np.sum(np.square(W3)))
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost

def backward_propagation_with_regularization(X, Y, cache, lambd): # cache is from forward_propagation()
# this is the implementation of backpropagation with L2 regularization as defined above  
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T) +(lambd/m)*W3
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T) +(lambd/m)*W2
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T) + (lambd/m)*W1
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

# finally, we can test our model with L2 regularization
parameters = model(train_X, train_Y, lambd = 0.7)
print ("With L2 regularization, on the training set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("With L2 regularization, on the test set:")
predictions_test = predict(test_X, test_Y, parameters)

# next, let's implement the necessary functions for dropout:
def forward_propagation_with_dropout(X, parameters, keep_prob = 0.5): # where keep_prob is the probability of keeping a neuron active during drop-out (scalar)
# Implements the forward propagation: LINEAR -> RELU + DROPOUT -> LINEAR -> RELU + DROPOUT -> LINEAR -> SIGMOID.
    
    # retrieve parameters
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    
    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)

    D1 = np.random.rand(A1.shape[0], A1.shape[1])    # Step 1: initialize matrix D1 = np.random.rand(..., ...), where D1 is a matrix containing 1s and 0s in the shape of A1
    D1 = D1 < keep_prob                              # Step 2: convert entries of D1 to 0 or 1 (using keep_prob as the threshold)
    A1 = A1*D1                                       # Step 3: shut down (eliminate) some neurons of A1
    A1 = A1/keep_prob                                # Step 4: scale the value of neurons that haven't been shut down

    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)

    D2 = np.random.rand(A2.shape[0], A1.shape[1])    # Step 1: initialize matrix D2 = np.random.rand(..., ...)
    D2 = D2 < keep_prob                              # Step 2: convert entries of D2 to 0 or 1 (using keep_prob as the threshold)
    A2 = A2*D2                                       # Step 3: shut down some neurons of A2
    A2 = A2/keep_prob                                # Step 4: scale the value of neurons that haven't been shut down

    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3)
    
    return A3, cache

def backward_propagation_with_dropout(X, Y, cache, keep_prob): # where cache is from forward_propagation_with_dropout    
    m = X.shape[1]
    (Z1, D1, A1, W1, b1, Z2, D2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1./m * np.dot(dZ3, A2.T)
    db3 = 1./m * np.sum(dZ3, axis=1, keepdims = True)
    dA2 = np.dot(W3.T, dZ3)

    dA2 = dA2*D2                # Step 1: Apply mask D2 to shut down the same neurons as during the forward propagation
    dA2 = dA2/keep_prob         # Step 2: Scale the value of neurons that haven't been shut down

    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)

    dA1 = dA1*D1               # Step 1: Apply mask D1 to shut down the same neurons as during the forward propagation
    dA1 = dA1/keep_prob        # Step 2: Scale the value of neurons that haven't been shut down

    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    gradients = {"dZ3": dZ3, "dW3": dW3, "db3": db3,"dA2": dA2,
                 "dZ2": dZ2, "dW2": dW2, "db2": db2, "dA1": dA1, 
                 "dZ1": dZ1, "dW1": dW1, "db1": db1}
    
    return gradients

# finally, let's test our 3-layer neural network with dropout 
parameters = model(train_X, train_Y, keep_prob = 0.86, learning_rate = 0.3)

print ("With dropout, on the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("With dropout, on the test set:")
predictions_test = predict(test_X, test_Y, parameters)